The purpose of this study is to propose a prediction model for 30-day unplanned readmission among diabetes patients in US hospitals. The analysis will be based on risk factors such as a patient’s demographics, admission details, diagnosis, and medical data. In a broader sense, the goal of the study is to allow health centers to better anticipate and address unplanned readmissions while improving their quality of care and cost efficiency.

### Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Import dataset

In [2]:
diabetes = pd.read_csv('diabetic_data.csv')

diabetes.head()

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


### Data exploration and Cleaning

Cleaning tasks such as dropping bad data, dealing with missing values.

In [3]:
diabetes.shape

(101766, 50)

In [4]:
diabetes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101766 entries, 0 to 101765
Data columns (total 50 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   encounter_id              101766 non-null  int64 
 1   patient_nbr               101766 non-null  int64 
 2   race                      101766 non-null  object
 3   gender                    101766 non-null  object
 4   age                       101766 non-null  object
 5   weight                    101766 non-null  object
 6   admission_type_id         101766 non-null  int64 
 7   discharge_disposition_id  101766 non-null  int64 
 8   admission_source_id       101766 non-null  int64 
 9   time_in_hospital          101766 non-null  int64 
 10  payer_code                101766 non-null  object
 11  medical_specialty         101766 non-null  object
 12  num_lab_procedures        101766 non-null  int64 
 13  num_procedures            101766 non-null  int64 
 14  num_

There are 50 features in this data. It conatins numerical values(int) and categorical values(object).

There are 101766 samples present in this dataset. There seem to be no null values.We will dig deep and find out more.

In [5]:
diabetes.describe()

,encounter_id,patient_nbr,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses
count,1.017660e+05,1.017660e+05,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000
mean,1.652016e+08,5.433040e+07,2.024006,3.715642,5.754437,4.395987,43.095641,1.339730,16.021844,0.369357,0.197836,0.635566,7.422607
std,1.026403e+08,3.869636e+07,1.445403,5.280166,4.064081,2.985108,19.674362,1.705807,8.127566,1.267265,0.930472,1.262863,1.933600
min,1.252200e+04,1.350000e+02,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000
25%,8.496119e+07,2.341322e+07,1.000000,1.000000,1.000000,2.000000,31.000000,0.000000,10.000000,0.000000,0.000000,0.000000,6.000000
50%,1.523890e+08,4.550514e+07,1.000000,1.000000,7.000000,4.000000,44.000000,1.000000,15.000000,0.000000,0.000000,0.000000,8.000000
75%,2.302709e+08,8.754595e+07,3.000000,4.000000,7.000000,6.000000,57.000000,2.000000,20.000000,0.000000,0.000000,1.000000,9.000000
max,4.438672e+08,1.895026e+08,8.000000,28.000000,25.000000,14.000000,132.000000,6.000000,81.000000,42.000000,76.000000,21.000000,16.000000


There are 50 features looking into all of these.

1. encounter_id : unique identifier of an encounter

In [6]:
diabetes['encounter_id'].nunique()

101766

All the patients that have come to the hospital have been provided with an encounter_id so that is why there are 101766 different unique values.

2. patient_nbr : unique identifier of a patient

In [7]:
diabetes['patient_nbr'].nunique()

71518

There are 71518 unique paitents that have entered the system.

encounter_id and patient_nbr: these are just identifiers and not useful variables so they can be dropped from our future analysis

In [8]:
diabetes.drop(['encounter_id','patient_nbr'],axis=1,inplace=True)

3. race : It is a categorical variable.

In [9]:
diabetes['race'].value_counts()

Caucasian          76099
AfricanAmerican    19210
?                   2273
Hispanic            2037
Other               1506
Asian                641
Name: race, dtype: int64

There are 5 distinct categories under race feature. There are some unknown values here entered in the form of '?' which needs to be replaced with nan values so that later these values can be filled.

In [10]:
diabetes['race'] = diabetes['race'].replace('?',np.nan)

4. gender: It is a categorical feature depicting the gender of the patient.

In [11]:
diabetes.gender.value_counts()

Female             54708
Male               47055
Unknown/Invalid        3
Name: gender, dtype: int64

There are 2 categories here: female, male. There is a 3rd category of unknown/invalid which are missing/unknown values ,as there count is less so they can be dropped.

In [12]:
diabetes['gender'] = diabetes['gender'].replace('Unknown/Invalid',np.nan)
diabetes['gender'].dropna(inplace=True)

In [13]:
diabetes.gender.value_counts()

Female    54708
Male      47055
Name: gender, dtype: int64

5. Age: It is a categorical feature as the age is grouped here in bins of 10 yrs

In [14]:
diabetes.age.value_counts()

[70-80)     26068
[60-70)     22483
[50-60)     17256
[80-90)     17197
[40-50)      9685
[30-40)      3775
[90-100)     2793
[20-30)      1657
[10-20)       691
[0-10)        161
Name: age, dtype: int64

The age here range from 0 to 100 yrs. 

The population falling in the range of 50 to 90 yrs are mostly present in this dataset.

6. Weight : weight in pounds.

In [15]:
diabetes.weight.value_counts()

?            98569
[75-100)      1336
[50-75)        897
[100-125)      625
[125-150)      145
[25-50)         97
[0-25)          48
[150-175)       35
[175-200)       11
>200             3
Name: weight, dtype: int64

Weight is also divided into bins of 25 pounds, but a significant part of the weight has '?' values. So it a good idea to drop this column.

In [16]:
diabetes.drop(['weight'],axis=1,inplace=True)

7. admission_type_id : Integer identifier corresponding to 8 distinct values, for example, emergency, urgent, elective, newborn, and not available

In [17]:
diabetes.admission_type_id.nunique()

8

There are 8 unique values ranging from 1 to 8.

In [18]:
diabetes.admission_type_id.value_counts()

1    53990
3    18869
2    18480
6     5291
5     4785
8      320
7       21
4       10
Name: admission_type_id, dtype: int64

These 8 values correspond to : 

1	Emergency
2	Urgent
3	Elective
4	Newborn
5	Not Available
6	NULL
7	Trauma Center
8	Not Mapped

From this we can see that the majority patients that have been admitted in this hospital have an admission type as emergency.

For further analysis we can merger these categories into less number of categories.

Emeregncy/Urgent/Trauma Center ---> Emergency 

Not Available/Null/Not Mapped ---> Not Available

Elective ---> Elective

New Born ---> New Born

In [19]:
diabetes['admission_type_id']=diabetes['admission_type_id'].replace({1:'Emergency',2:'Emergency',7:'Emergency',
                                 5:'Not Available', 6:'Not Available', 8:'Not Available',
                                 3:'Elective',4:'Newborn'})

In [20]:
diabetes.admission_type_id.value_counts()

Emergency        72491
Elective         18869
Not Available    10396
Newborn             10
Name: admission_type_id, dtype: int64

Droppping the category new born because the admission_type_id == newborn and the age corresponding to it give contradictory information.

In [21]:
diabetes=diabetes[diabetes['admission_type_id']!='Newborn']

In [22]:
diabetes.admission_type_id.value_counts()

Emergency        72491
Elective         18869
Not Available    10396
Name: admission_type_id, dtype: int64

8. Discharge disposition: Integer identifier corresponding to 26 distinct values, for example, discharged to home, expired, and not available

In [23]:
diabetes.discharge_disposition_id.value_counts()

1     60228
3     13954
6     12898
18     3691
2      2128
22     1993
11     1642
5      1184
25      989
4       815
7       623
23      412
13      399
14      372
28      139
8       108
15       63
24       48
9        21
17       14
16       11
19        8
10        6
27        5
12        3
20        2
Name: discharge_disposition_id, dtype: int64

#### discharge_disposition_id	description

1	Discharged to home

2	Discharged/transferred to another short term hospital

3	Discharged/transferred to SNF(skilled nursing facility)

4	Discharged/transferred to ICF(intermediate care facility)

5	Discharged/transferred to another type of inpatient care institution

6	Discharged/transferred to home with home health service

7	Left AMA(Against Medical Advice.)

8	Discharged/transferred to home under care of Home IV provider

9	Admitted as an inpatient to this hospital

10	Neonate discharged to another hospital for neonatal aftercare

11	Expired

12	Still patient or expected to return for outpatient services

13	Hospice / home

14	Hospice / medical facility

15	Discharged/transferred within this institution to Medicare approved swing bed

16	Discharged/transferred/referred another institution for outpatient services

17	Discharged/transferred/referred to this institution for outpatient services

18	NULL

19	Expired at home. Medicaid only, hospice.

20	Expired in a medical facility. Medicaid only, hospice.

21	Expired, place unknown. Medicaid only, hospice.

22	Discharged/transferred to another rehab fac including rehab units of a hospital .

23	Discharged/transferred to a long term care hospital.

24	Discharged/transferred to a nursing facility certified under Medicaid but not certified under Medicare.

25	Not Mapped

26	Unknown/Invalid

30	Discharged/transferred to another Type of Health Care Institution not Defined Elsewhere

27	Discharged/transferred to a federal health care facility.

28	Discharged/transferred/referred to a psychiatric hospital of psychiatric distinct part unit of a hospital

29	Discharged/transferred to a Critical Access Hospital (CAH).

#### We can merge several grps over here to reduce the number of categories:

1 ---> Discharged to home

2/3/4/5/10/16/22/23/24/30/27/28/29 ---> Transferred to another medical facility

7 ---> Left AMA(Against Medical Advice.)

6/8 ---> Discharged to home with home health service

9/12/15/17 ---> Still patient/referred to this institution

11/19/20/21  ---> Expired

18/25/26 ---> Not Available

13/14 ---> Hospice

In [24]:
diabetes['discharge_disposition_id']=diabetes['discharge_disposition_id'].replace({1:'Discharged to home',
                                        2:'Transferred to another medical facility',
                                        3:'Transferred to another medical facility',
                                        4:'Transferred to another medical facility',
                                        5:'Transferred to another medical facility',
                                        16:'Transferred to another medical facility',
                                        22:'Transferred to another medical facility',
                                        23:'Transferred to another medical facility',
                                        24:'Transferred to another medical facility',
                                        27:'Transferred to another medical facility',
                                        28:'Transferred to another medical facility',
                                        29:'Transferred to another medical facility',
                                        30:'Transferred to another medical facility',                            
                                        6:'Discharged to home with home health service',
                                        8:'Discharged to home with home health service',
                                        7:'Left AMA',
                                        9:'Still patient/referred to this institution',
                                        12:'Still patient/referred to this institution',
                                        15:'Still patient/referred to this institution',
                                        17:'Still patient/referred to this institution',
                                        10:'Neonate discharged',
                                        11:'Expired',
                                        19:'Expired',
                                        20:'Expired',
                                        21:'Expired',
                                        13:'Hospice',
                                        14:'Hospice',
                                        18:'Not Available',
                                        25:'Not Available',
                                        26:'Not Available'                                 
})

In [25]:
diabetes.discharge_disposition_id.value_counts()

Discharged to home                             60228
Transferred to another medical facility        20689
Discharged to home with home health service    13006
Not Available                                   4680
Expired                                         1652
Hospice                                          771
Left AMA                                         623
Still patient/referred to this institution       101
Neonate discharged                                 6
Name: discharge_disposition_id, dtype: int64

The patients who have expired should be removed from the system beacuse they will create bias as they will not be readmitted.

In [26]:
diabetes=diabetes[diabetes['discharge_disposition_id']!='Expired']
diabetes=diabetes[diabetes['discharge_disposition_id']!='Neonate discharged']

In [27]:
diabetes.discharge_disposition_id.value_counts()

Discharged to home                             60228
Transferred to another medical facility        20689
Discharged to home with home health service    13006
Not Available                                   4680
Hospice                                          771
Left AMA                                         623
Still patient/referred to this institution       101
Name: discharge_disposition_id, dtype: int64

9. admission_source_id: Integer identifier corresponding to 17 distinct values, for example, physician referral, emergency room, and transfer from a hospital

In [28]:
diabetes.admission_source_id.value_counts(sort=False)

1     29317
2      1083
3       185
4      3127
5       814
6      2244
7     56358
8        15
9       125
10        8
11        2
13        1
14        1
17     6645
20      159
22       12
25        2
Name: admission_source_id, dtype: int64


1	 Physician Referral

2	Clinic Referral

3	HMO Referral(Health Maintenance Organization)

4	Transfer from a hospital

5	 Transfer from a Skilled Nursing Facility (SNF)

6	 Transfer from another health care facility

7	 Emergency Room

8	 Court/Law Enforcement

9	 Not Available

10	 Transfer from critial access hospital

11	Normal Delivery

12	 Premature Delivery

13	 Sick Baby

14	 Extramural Birth

15	Not Available

17	NULL

18	 Transfer From Another Home Health Agency

19	Readmission to Same Home Health Agency

20	 Not Mapped

21	Unknown/Invalid

22	 Transfer from hospital inpt/same fac reslt in a sep claim

23	 Born inside this hospital

24	 Born outside this hospital

25	 Transfer from Ambulatory Surgery Center

26	Transfer from Hospice

#### We can merge several grps over here to reduce the number of categories:

1/2/3 ---> Referral

4/5/6/10/18/19/25/26/22 ---> Transferred from another health care facility

7 ---> Emergency

9/15/17/20/21 ---> Not Available

8 --->  Court/Law Enforcement

Delivery ---> 11/12/14/13/23/24

In [29]:
diabetes['admission_source_id']=diabetes['admission_source_id'].replace({1:'Referral',
                                   2:'Referral',
                                   3:'Referral',
                                   4:'Transferred from another health care facility',
                                   5:'Transferred from another health care facility',
                                   6:'Transferred from another health care facility',
                                   8:'Court/Law Enforcement',                                    
                                   10:'Transferred from another health care facility',
                                   18:'Transferred from another health care facility',
                                   19:'Transferred from another health care facility',
                                   25:'Transferred from another health care facility',
                                   26:'Transferred from another health care facility',
                                   7:'Emergency',
                                   9:'Not Available',
                                   15:'Not Available',
                                   17:'Not Available',
                                   20:'Not Available',
                                   21:'Not Available',
                                   11:'Delivery',
                                   12:'Delivery',
                                   13:'Delivery',
                                   14:'Delivery',
                                   23:'Delivery',
                                   22:"Transferred from another health care facility",                                    
                                   24:'Delivery'                                

                                   
})

In [30]:
diabetes.admission_source_id.value_counts()

Emergency                                        56358
Referral                                         30585
Not Available                                     6929
Transferred from another health care facility     6207
Court/Law Enforcement                               15
Delivery                                             4
Name: admission_source_id, dtype: int64

Droppping the category delivery  because the admission_source_id == delivery and the age corresponding to it give contradictory information.

In [31]:
diabetes=diabetes[diabetes['admission_source_id']!='Delivery']

In [32]:
diabetes.admission_source_id.value_counts()

Emergency                                        56358
Referral                                         30585
Not Available                                     6929
Transferred from another health care facility     6207
Court/Law Enforcement                               15
Name: admission_source_id, dtype: int64

10. Time in hospital: Integer number of days between admission and discharge

In [33]:
diabetes.time_in_hospital.value_counts(sort=False)

1     13878
2     16971
3     17539
4     13773
5      9832
6      7424
7      5744
8      4321
9      2926
10     2290
11     1809
12     1401
13     1173
14     1013
Name: time_in_hospital, dtype: int64

The time in the hospital ranges from 1 day to 14 days.

11. payer_code : Integer identifier corresponding to 23 distinct values, for example, Blue Cross/Blue Shield, Medicare, and self-pay Medical

In [34]:
diabetes.payer_code.value_counts(sort=False)

MC    31738
CH      144
WC      135
SI       55
BC     4625
MP       79
FR        1
OG     1023
CP     2496
HM     6218
DM      546
CM     1903
?     39580
SP     4956
PO      586
OT       94
UN     2424
MD     3491
Name: payer_code, dtype: int64

Payer code has a significant number of null values ('?') and it are not signification w.r.t the patient readmission so we can drop it. 

In [35]:
diabetes.drop(['payer_code'],axis=1,inplace=True)

12.medical_specialty: Integer identifier of a specialty of the admitting physician, corresponding to 73 distinct values, for example, cardiology, internal medicine, family/general practice, and surgeon

In [36]:
diabetes.medical_specialty.nunique()

73

In [37]:
diabetes['medical_specialty'] = diabetes['medical_specialty'].replace('?',np.nan)

13. num_lab_procedures: Number of lab tests performed during the encounter

In [38]:
print('The number of lab tests a patient has undergone can range from',diabetes.num_lab_procedures.min(),'to',diabetes.num_lab_procedures.max())

The number of lab tests a patient has undergone can range from 1 to 132


14. num_procedures: Number of procedures(other than lab tests) performed during the encounter 

In [39]:
diabetes.num_procedures.value_counts()

0    46083
1    20438
2    12455
3     9245
6     4809
4     4067
5     2997
Name: num_procedures, dtype: int64

No of procedures range from 0 to 5

15. num_medications: Number of distinct generic names administered during the encounter

In [40]:
print('The number of medications range from a minimum of',diabetes.num_medications.min(),'to maximum of',diabetes.num_medications.max())

The number of medications range from a minimum of 1 to maximum of 81


16. number_outpatient: Number of outpatient visits of the patient in the year preceding the encounter 

17. number_emergency : Number of emergency visits of the patient in the year preceding the encounter

18. number_inpatient: Number of inpatient visits of the patient in the year preceding the encounter 

The above three measures depict the utilization of hospital/clinic services of the current patient in the past years, so we combined these features into a new variable called preceding_year_visits.

In [41]:
diabetes['preceding_year_visits'] = diabetes['number_outpatient'] + diabetes['number_emergency'] + diabetes['number_inpatient']

In [42]:
diabetes.drop(['number_outpatient','number_emergency','number_inpatient'],axis=1,inplace=True)

We noticed that for two variables (drugs named citoglipton and examide), all records have the same value. So essentially these cannot provide any interpretive for predicting readmission, so we dropped these columns.

In [43]:
diabetes.drop(['citoglipton', 'examide'], axis = 1,inplace=True)

#### Outcome variable

The outcome we are looking at is whether the patient gets readmitted to the hospital within 30 days or not.

In [44]:
diabetes.readmitted.value_counts()

NO     53199
>30    35539
<30    11356
Name: readmitted, dtype: int64

The outcome actually has < 30, > 30 and No Readmission categories.So wrt our problem we will convert this into 2 categories i.e No readmission or readmission with in 30 days

In [45]:
diabetes['readmitted'] = diabetes['readmitted'].replace('>30', 'NO')

In [46]:
diabetes.readmitted.value_counts()

NO     88738
<30    11356
Name: readmitted, dtype: int64

In [47]:
diabetes.head()

,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,medical_specialty,num_lab_procedures,num_procedures,...,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted,preceding_year_visits
0,Caucasian,Female,[0-10),Not Available,Not Available,Referral,1,Pediatrics-Endocrinology,41,0,...,No,No,No,No,No,No,No,No,NO,0
1,Caucasian,Female,[10-20),Emergency,Discharged to home,Emergency,3,NaN,59,0,...,Up,No,No,No,No,No,Ch,Yes,NO,0
2,AfricanAmerican,Female,[20-30),Emergency,Discharged to home,Emergency,2,NaN,11,5,...,No,No,No,No,No,No,No,Yes,NO,3
3,Caucasian,Male,[30-40),Emergency,Discharged to home,Emergency,2,NaN,44,1,...,Up,No,No,No,No,No,Ch,Yes,NO,0
4,Caucasian,Male,[40-50),Emergency,Discharged to home,Emergency,1,NaN,51,0,...,Steady,No,No,No,No,No,Ch,Yes,NO,0


In [48]:
diabetes.shape

(100094, 42)

In [49]:
df = diabetes.reset_index(drop = True )

In [50]:
diag_cols = ['diag_1','diag_2','diag_3']

In [51]:
ord('E'), ord('V')

(69, 86)

In [52]:
df['diag_1'].value_counts()

428       6735
414       6552
786       4015
410       3477
486       3413
          ... 
974          1
V60          1
885          1
250.51       1
523          1
Name: diag_1, Length: 715, dtype: int64

In [ ]:
for i in diag_cols:
    for j in range(len(df[i])):
        if str(df.loc[j, i])[:3] == '250':
            df.loc[j, i] = 'Diabetes'
        elif ord(str(df.loc[j, i])[0]) in range(69, 87):
            df.loc[j, i] = 'External causes of injury'
        elif df.loc[j, i] == '?':
            df.loc[j, i] = 'Missing'
        else:
            x = float(df.loc[j, i])
            if x in range(390, 460) or x == 785:
                df.loc[j, i] = 'Circulatory'
            elif x in range(460, 520) or x == 786:
                df.loc[j, i] = 'Respiratory'
            elif x in range(520, 580) or x == 787:
                df.loc[j, i] = 'Digestive'
            elif x in range(800, 1000):
                df.loc[j, i] = 'Injury and Poisoning'
            elif x in range(710, 740):
                df.loc[j, i] = 'Musculoskeletal System and Connective Tissue'
            elif x in range(580, 630) or x == 788:
                df.loc[j, i] = 'Genitourinary'
            elif x in range(140, 230):
                df.loc[j, i] = 'Neoplasms'
            elif x in [780, 781, 784] or x in range(790, 800):
                df.loc[j, i] = 'Other Symptoms'
            elif x in range(240, 280):
                df.loc[j, i] = 'Endocrine, Nutritional, Metabolic, Immunity'
            elif x in range(680, 710) or x == 782:
                df.loc[j, i] = 'Skin and Subcutaneous Tissue'
            elif x in range(1, 140):
                df.loc[j, i] = 'Infectious and Parasitic'
            elif x in range(290, 320):
                df.loc[j, i] = 'Mental Disorders'
            elif x in range(280, 290):
                df.loc[j, i] = 'Blood and Blood-Forming Organs'
            elif x in range(320, 360):
                df.loc[j, i] = 'Nervous'
            elif x in range(630, 680):
                df.loc[j, i] = 'Pregnancy, Childbirth'
            elif x in range(360, 390):
                df.loc[j, i] = 'Sense Organs'
            else:
                df.loc[j, i] = 'Congenital Anomalies'

In [54]:
df['diag_1'].value_counts()

Circulatory                                     29874
Respiratory                                     14072
Digestive                                        9379
Diabetes                                         8692
Injury and Poisoning                             6878
Genitourinary                                    5054
Musculoskeletal System and Connective Tissue     4941
Neoplasms                                        3134
Endocrine, Nutritional, Metabolic, Immunity      2667
Skin and Subcutaneous Tissue                     2597
Infectious and Parasitic                         2585
Other Symptoms                                   2526
Mental Disorders                                 2259
External causes of injury                        1638
Blood and Blood-Forming Organs                   1095
Nervous                                           928
Congenital Anomalies                              804
Pregnancy, Childbirth                             686
Sense Organs                

In [55]:
df['diag_2'].value_counts()

Circulatory                                     31363
Diabetes                                        12720
Respiratory                                     10504
Genitourinary                                    8232
Endocrine, Nutritional, Metabolic, Immunity      8113
Digestive                                        4114
Skin and Subcutaneous Tissue                     3639
Blood and Blood-Forming Organs                   2889
Mental Disorders                                 2648
External causes of injury                        2523
Neoplasms                                        2418
Other Symptoms                                   2395
Injury and Poisoning                             2389
Infectious and Parasitic                         1868
Musculoskeletal System and Connective Tissue     1762
Nervous                                          1085
Congenital Anomalies                              500
Pregnancy, Childbirth                             414
Missing                     

In [56]:
df['diag_3'].value_counts()

Circulatory                                     29782
Diabetes                                        17021
Endocrine, Nutritional, Metabolic, Immunity      8974
Respiratory                                      7096
Genitourinary                                    6519
External causes of injury                        5033
Digestive                                        3885
Mental Disorders                                 3116
Skin and Subcutaneous Tissue                     2570
Blood and Blood-Forming Organs                   2462
Other Symptoms                                   2345
Injury and Poisoning                             1904
Musculoskeletal System and Connective Tissue     1903
Infectious and Parasitic                         1825
Neoplasms                                        1719
Nervous                                          1460
Missing                                          1421
Congenital Anomalies                              489
Pregnancy, Childbirth       

In [57]:
df = df[['race', 'gender', 'age', 'admission_type_id',
       'discharge_disposition_id', 'admission_source_id', 'time_in_hospital',
       'medical_specialty', 'num_lab_procedures', 'num_procedures',
       'num_medications', 'preceding_year_visits', 'diag_1', 'diag_2', 'diag_3', 'number_diagnoses',
       'max_glu_serum', 'A1Cresult', 'metformin', 'repaglinide', 'nateglinide',
       'chlorpropamide', 'glimepiride', 'acetohexamide', 'glipizide',
       'glyburide', 'tolbutamide', 'pioglitazone', 'rosiglitazone', 'acarbose',
       'miglitol', 'troglitazone', 'tolazamide', 'insulin',
       'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed', 'readmitted']]

In [58]:
df1 = pd.DataFrame(df['diag_1'].value_counts())
df2 = df1.merge(pd.DataFrame(df['diag_2'].value_counts()), how = 'outer', left_index = True, right_index = True)
df2.merge(pd.DataFrame(df['diag_3'].value_counts()), how = 'outer', left_index = True, right_index = True)

,diag_1,diag_2,diag_3
Blood and Blood-Forming Organs,1095,2889,2462
Circulatory,29874,31363,29782
Congenital Anomalies,804,500,489
Diabetes,8692,12720,17021
Digestive,9379,4114,3885
"Endocrine, Nutritional, Metabolic, Immunity",2667,8113,8974
External causes of injury,1638,2523,5033
Genitourinary,5054,8232,6519
Infectious and Parasitic,2585,1868,1825
Injury and Poisoning,6878,2389,1904


In [59]:
for i in range(len(df['diag_2'])):
    if df.loc[i, 'diag_2'] == 'Missing':
        if df.loc[i, 'diag_1'] != 'Missing':
            df.loc[i, 'diag_2'] = 'Not Required'

In [60]:
for i in range(len(df['diag_3'])):
    if df.loc[i, 'diag_3'] == 'Missing':
        if df.loc[i, 'diag_2'] == 'Not Required' or df.loc[i, 'diag_2'] != 'Missing':
            df.loc[i, 'diag_3'] = 'Not Required'

In [61]:
df[df['diag_2'] == 'Missing']

,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,medical_specialty,num_lab_procedures,num_procedures,...,tolazamide,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
85717,Asian,Male,[60-70),Emergency,Hospice,Transferred from another health care facility,7,InternalMedicine,1,2,...,No,No,No,No,No,No,No,No,No,NO


In [62]:
df.drop(85717, inplace = True)
df.shape

(100093, 42)

In [63]:
df1 = pd.DataFrame(df['diag_1'].value_counts())
df2 = df1.merge(pd.DataFrame(df['diag_2'].value_counts()), how = 'outer', left_index = True, right_index = True)
df2.merge(pd.DataFrame(df['diag_3'].value_counts()), how = 'outer', left_index = True, right_index = True)

,diag_1,diag_2,diag_3
Blood and Blood-Forming Organs,1095.0,2889.0,2462.0
Circulatory,29874.0,31363.0,29782.0
Congenital Anomalies,804.0,500.0,489.0
Diabetes,8692.0,12720.0,17021.0
Digestive,9379.0,4114.0,3885.0
"Endocrine, Nutritional, Metabolic, Immunity",2667.0,8113.0,8974.0
External causes of injury,1638.0,2523.0,5033.0
Genitourinary,5054.0,8232.0,6519.0
Infectious and Parasitic,2585.0,1868.0,1825.0
Injury and Poisoning,6878.0,2389.0,1904.0
